## TFRecord 만들기 : 각각의 파일로

In [23]:
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm.notebook import tqdm

from glob import glob
import random

import warnings

warnings.filterwarnings(action='ignore')

############ Tensorflow에서 제공된 Type별 Feature 생성 코드 ############

def _bytes_feature(value):
    # string / byte 타입을 받아서 byte list를 리턴.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    # float / double 타입을 받아서 float list를 리턴
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _floatarray_feature(array):
    # float / double 타입을 받아서 float list를 리턴
    return tf.train.Feature(float_list=tf.train.FloatList(value=array))

def _int64_feature(value):
    # bool / enum / int / uint 타입을 받아서 int64 list를 리턴
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

'''label 별로 만들어줄거임'''
def to_example(filename, label):
    
    # 원본 이미지를 resize하지 않고 TFRecord로 저장.
    image_string = tf.io.read_file(filename)    
    
    shape = tf.image.decode_png(image_string).shape

    feature = {
        'image/height': _int64_feature(shape[0]),
        'image/width': _int64_feature(shape[1]),
        'image/channel': _int64_feature(shape[2]),
        'image/label': _int64_feature(label),
        'image/image_raw': _bytes_feature(image_string),
        'image/filename': _bytes_feature(filename.encode())        
    }
    
    # Example 객체 생성
    # 파이썬의 문자열은 모두 unicode. unicode 문자열을 byte배열로 바꾸는 함수가 encode()
    # 각 type에 맞게 Feature 객체 생성 후 dict 생성. 
    # 이 dict를 이용해 Feauture 객체 생성 후 Example 객체 생성.
    return tf.train.Example(features=tf.train.Features(feature=feature))

def chunkify(filename_list, n):
    size = len(filename_list) // n
    start = 0
    results = []
    for i in range(n - 1):
        results.append(filename_list[start:start + size])
        start += size
    results.append(filename_list[start:])
    return results

def build_tfrecords(total_shards_num, split, filenames, label):
    chunks = chunkify(filenames, total_shards_num)
    failed = 0
    for i, chunk in tqdm(enumerate(chunks),
                         total=len(chunks),
                         position=0,
                         leave=True):        
        tfrecords_path = './tfrecords_0428_1/{}/{}_{}_of_{}.tfrecords'.format(split,label,
                                                                    str(i + 1).zfill(4),
                                                                    str(total_shards_num).zfill(4))
        # GZIP으로 압축한 TFRecord 생성하기 위한 option
        # options = tf.io.TFRecordOptions(compression_type='GZIP')
        
        # with tf.io.TFRecordWriter(tfrecords_path, options=options) as writer:
        with tf.io.TFRecordWriter(tfrecords_path) as writer:
            for filename in chunk:
                try:
                    tf_example = to_example(filename, label)
                    
                    # 만들어진 Example 객체를 binary string으로 변환한 후 파일에 저장
                    writer.write(tf_example.SerializeToString())
                except:
                    print(f'fail: {filename}')

In [2]:
path_dir = './data/train_ex'
file_list = os.listdir(path_dir)
file_list

['bottle-broken_large',
 'bottle-broken_small',
 'bottle-contamination',
 'bottle-good',
 'cable-bent_wire',
 'cable-cable_swap',
 'cable-combined',
 'cable-cut_inner_insulation',
 'cable-cut_outer_insulation',
 'cable-good',
 'cable-missing_cable',
 'cable-missing_wire',
 'cable-poke_insulation',
 'capsule-crack',
 'capsule-faulty_imprint',
 'capsule-good',
 'capsule-poke',
 'capsule-scratch',
 'capsule-squeeze',
 'carpet-color',
 'carpet-cut',
 'carpet-good',
 'carpet-hole',
 'carpet-metal_contamination',
 'carpet-thread',
 'grid-bent',
 'grid-broken',
 'grid-glue',
 'grid-good',
 'grid-metal_contamination',
 'grid-thread',
 'hazelnut-crack',
 'hazelnut-cut',
 'hazelnut-good',
 'hazelnut-hole',
 'hazelnut-print',
 'leather-color',
 'leather-cut',
 'leather-fold',
 'leather-glue',
 'leather-good',
 'leather-poke',
 'metal_nut-bent',
 'metal_nut-color',
 'metal_nut-flip',
 'metal_nut-good',
 'metal_nut-scratch',
 'pill-color',
 'pill-combined',
 'pill-contamination',
 'pill-crack',
 'p

In [24]:
os.makedirs('tfrecords_0428_1', exist_ok=True)
os.makedirs('tfrecords_0428_1/train', exist_ok=True)
os.makedirs('tfrecords_0428_1/valid', exist_ok=True)

label = 0

for f_name in file_list:
    
    filenames = tf.io.gfile.glob('./data/train_ex/'+f_name+'/*.png')

    train_data, valid_data = train_test_split(filenames, train_size=0.8)
#     print(f_name,'================================')
#     print(train_data)
#     print(valid_data)
    build_tfrecords(len(train_data), 'train', train_data, label)
    build_tfrecords(len(valid_data), 'valid', valid_data, label)
    
    label += 1

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]